In [1]:
# Shabat, IUM, spring 2019, Algebra-2 course, excercises 5.2 and 5.3
# Generic code for displaying multiplication tables
import itertools as it
from IPython.core.display import Latex, Math
import sympy as sp
import pandas as pd
from tabulate import tabulate

In [2]:
# sp.init_printing()

In [3]:
x = sp.symbols("x")

In [4]:
def zero_polynomial(p):
    F_p = sp.FF(p)
    return sp.Poly([], x, domain=F_p)


def iter_all_polynomials(p, deg):
    F_p = sp.FF(p)
    all_scalars = [F_p(n) for n in range(p)]
    powers = [x**n for n in range(deg + 1)]
    for coeff_tuple in it.product(all_scalars, repeat=deg + 1):
        if coeff_tuple[0] == 0:
            continue
        yield sp.Poly(coeff_tuple, x, domain=F_p)


def expr_product(exprs):
    res = exprs[0].as_expr()
    for expr in exprs[1:]:
        res *= expr.as_expr()
    return res


def iter_all_polynomials_with_factorings(p, max_deg):
    table = {}
    for f in iter_all_polynomials(p, 1):
        table[f] = (f,)
        yield f, (f,)
    for deg in range(2, max_deg + 1):
        for f, f_factors in list(table.items()):
            for g, g_factors in list(table.items()):
                h = f * g
                if h.degree() == deg:
                    table[h] = (*f_factors, *g_factors)
        for f in iter_all_polynomials(p, deg):
            f_factors = table.setdefault(f, (f, ))
            yield f, f_factors
            

def return_dict(generator):
    def func(*args, **kwargs):
        return dict(generator(*args, **kwargs))
    return func


@return_dict
def as_table(polynomials_with_factorings):
    for f, f_factors in polynomials_with_factorings:
        yield f.as_expr(), expr_product(f_factors)


def sympy_to_latex(v):
    return "$" + sp.latex(v) + "$"


display(pd.Series(as_table(iter_all_polynomials_with_factorings(2, 3))))
display(pd.Series(as_table(iter_all_polynomials_with_factorings(3, 2))))

x                                           x
x + 1                                   x + 1
x**2                                     x**2
x**2 + 1                           (x + 1)**2
x**2 + x                            x*(x + 1)
x**2 + x + 1                     x**2 + x + 1
x**3                                     x**3
x**3 + 1               (x + 1)*(x**2 + x + 1)
x**3 + x                         x*(x + 1)**2
x**3 + x + 1                     x**3 + x + 1
x**3 + x**2                      x**2*(x + 1)
x**3 + x**2 + 1               x**3 + x**2 + 1
x**3 + x**2 + x              x*(x**2 + x + 1)
x**3 + x**2 + x + 1                (x + 1)**3
dtype: object

x                               x
x + 1                       x + 1
x - 1                       x - 1
-x                             -x
1 - x                       1 - x
-x - 1                     -x - 1
x**2                         x**2
x**2 + 1                 x**2 + 1
x**2 - 1         (1 - x)*(-x - 1)
x**2 + x              -x*(-x - 1)
x**2 + x + 1           (1 - x)**2
x**2 + x - 1         x**2 + x - 1
x**2 - x               -x*(1 - x)
x**2 - x + 1          (-x - 1)**2
x**2 - x - 1         x**2 - x - 1
-x**2                       -x**2
1 - x**2         (-x - 1)*(x - 1)
-x**2 - 1               -x**2 - 1
-x**2 + x               x*(1 - x)
-x**2 + x + 1       -x**2 + x + 1
-x**2 + x - 1    (-x - 1)*(x + 1)
-x**2 - x              x*(-x - 1)
-x**2 - x + 1       -x**2 - x + 1
-x**2 - x - 1     (1 - x)*(x - 1)
dtype: object

In [7]:
def elements_and_multiplication_table(p, f):
    f = sp.Poly(f, domain=sp.FF(p))
    deg = f.degree()
    elements = list(it.chain([zero_polynomial(p)], *[iter_all_polynomials(p, elem_deg) for elem_deg in range(deg)]))
    return [g.as_expr() for g in elements], [[(g * h % f).as_expr() for g in elements] for h in elements]


def as_matrix(e, m):
    return pd.DataFrame(m, columns=list(map(sympy_to_latex, e)), index=list(map(sympy_to_latex, e))).applymap(sympy_to_latex)

e, m = elements_and_multiplication_table(2, x**3 + x**2 + 1)
display(as_matrix(e, m))


e, m = elements_and_multiplication_table(3, x**2 - x - 1)
display(as_matrix(e, m))

,$0$,$1$,$x$,$x + 1$,$x^{2}$,$x^{2} + 1$,$x^{2} + x$,$x^{2} + x + 1$
$0$,$0$,$0$,$0$,$0$,$0$,$0$,$0$,$0$
$1$,$0$,$1$,$x$,$x + 1$,$x^{2}$,$x^{2} + 1$,$x^{2} + x$,$x^{2} + x + 1$
$x$,$0$,$x$,$x^{2}$,$x^{2} + x$,$x^{2} + 1$,$x^{2} + x + 1$,$1$,$x + 1$
$x + 1$,$0$,$x + 1$,$x^{2} + x$,$x^{2} + 1$,$1$,$x$,$x^{2} + x + 1$,$x^{2}$
$x^{2}$,$0$,$x^{2}$,$x^{2} + 1$,$1$,$x^{2} + x + 1$,$x + 1$,$x$,$x^{2} + x$
$x^{2} + 1$,$0$,$x^{2} + 1$,$x^{2} + x + 1$,$x$,$x + 1$,$x^{2} + x$,$x^{2}$,$1$
$x^{2} + x$,$0$,$x^{2} + x$,$1$,$x^{2} + x + 1$,$x$,$x^{2}$,$x + 1$,$x^{2} + 1$
$x^{2} + x + 1$,$0$,$x^{2} + x + 1$,$x + 1$,$x^{2}$,$x^{2} + x$,$1$,$x^{2} + 1$,$x$


,$0$,$1$,$-1$,$x$,$x + 1$,$x - 1$,$- x$,$1 - x$,$- x - 1$
$0$,$0$,$0$,$0$,$0$,$0$,$0$,$0$,$0$,$0$
$1$,$0$,$1$,$-1$,$x$,$x + 1$,$x - 1$,$- x$,$1 - x$,$- x - 1$
$-1$,$0$,$-1$,$1$,$- x$,$- x - 1$,$1 - x$,$x$,$x - 1$,$x + 1$
$x$,$0$,$x$,$- x$,$x + 1$,$1 - x$,$1$,$- x - 1$,$-1$,$x - 1$
$x + 1$,$0$,$x + 1$,$- x - 1$,$1 - x$,$-1$,$x$,$x - 1$,$- x$,$1$
$x - 1$,$0$,$x - 1$,$1 - x$,$1$,$x$,$- x - 1$,$-1$,$x + 1$,$- x$
$- x$,$0$,$- x$,$x$,$- x - 1$,$x - 1$,$-1$,$x + 1$,$1$,$1 - x$
$1 - x$,$0$,$1 - x$,$x - 1$,$-1$,$- x$,$x + 1$,$1$,$- x - 1$,$x$
$- x - 1$,$0$,$- x - 1$,$x + 1$,$x - 1$,$1$,$- x$,$1 - x$,$x$,$-1$
